In [11]:
from datasets import load_from_disk, concatenate_datasets
from transformers import MimiModel, AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("kyutai/mimi")
model = MimiModel.from_pretrained("kyutai/mimi")
model = model.to("mps").eval()

/Users/ritsuko/projects/ai/audio/dual-ar/.venv/lib/python3.9/site-packages/transformers/models/mimi/modeling_mimi.py:164: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


NOTE: dataset creation script is elsewhere.

If you are Jacob Keisling, please create a file `.env` in this repo and put your HuggingFace token under the `HUGGINGFACE_TOKEN=sk-xxxx` variable. Since you are probably not Jacob Keisling, if you want to re-upload, please pick a repo you actually own.

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

# If you created this from scratch
dataset = load_from_disk("./encoded_libritts")
dataset = dataset.with_format("torch")
# Uncomment if this is your first time pushing your new dataset
dataset.push_to_hub("jkeisling/libritts-r-mimi", token=os.getenv("HUGGINGFACE_TOKEN"))

Uploading the dataset shards: 100%|██████████| 2/2 [00:04<00:00,  2.16s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/jkeisling/libritts-r-mimi/commit/a7737c6e64509c17e366299a1597c58ee66dec10', commit_message='Upload dataset', commit_description='', oid='a7737c6e64509c17e366299a1597c58ee66dec10', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jkeisling/libritts-r-mimi', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jkeisling/libritts-r-mimi'), pr_revision=None, pr_num=None)

In [8]:
import torch
import torchaudio
from datasets import load_dataset
from dotenv import load_dotenv
import os

load_dotenv()
dataset = load_dataset("jkeisling/project-gutenberg-kokoro-2K", token=os.getenv("HUGGINGFACE_ACCESS_TOKEN"), streaming=False)
dataset = dataset.with_format("torch")
codes = dataset["train"][0]['codes']
codes

tensor([[1049,  127, 1880, 1031, 1031, 1031, 1492, 1492, 1926, 1926, 1826, 1268,
         1001,  382,  587,  178, 1380, 1380, 1790,  117,  130,  531, 2036,  722,
         1470, 1725,  371,  728,  774, 1677,  518,  769,  666,   84,   84,  752,
          752,  752,  752,  752, 1926, 1926, 1926, 1926],
        [1700,  243,  243,  243,  243,  243, 1211,  243,  243,  243,  832, 1920,
         1877, 1227, 1534,  860, 1576, 1984,  918, 1014, 1312, 2017,  104,  767,
          605,  373, 1494,   93, 1284,  449, 1168, 1700,  243,  243,  243,  243,
          243,  243,  243,  243,  243,  243,  243,  243],
        [1626,  783, 1178, 1178, 1178, 1178, 1697, 1178, 1178, 1559,  783, 1989,
         1982,   51, 1876, 1446, 1434, 1580,  256,   13,  470,  600,  105, 1936,
         1727,  102, 1859,  487,  926,  959, 1033, 1178, 1178,  783, 1178, 1178,
         1559, 1559, 1559, 1559, 1178, 1178, 1178, 1178],
        [ 546,  546,  546,  290,  142,  164,  546,  290,  290,  164, 1225, 1911,
          838,  

In [13]:
codes_input = codes.to('mps').to(torch.long).unsqueeze(0)
out_pcm = model.eval().decode(codes_input)
torchaudio.save("out.wav", out_pcm.audio_values[0].detach().to("cpu"), 24000)

In [19]:
def add_length(example):
    # Use .size(-1) if the code is a torch tensor; if it’s a list, use len()
    example["audio_length"] = int(example["codes"].size(-1))
    return example

dataset = dataset.map(add_length, num_proc=6)

Map (num_proc=6): 100%|██████████| 576820/576820 [00:28<00:00, 20049.86 examples/s]


In [ ]:
sum(dataset['train']['audio_length']) / 12.5 / 60 / 60

tensor(1291.7410)

In [22]:
dataset.column_names

{'train': ['text_normalized',
  'codes',
  'accent',
  'gender',
  'speaker_id',
  'audio_length']}